Neural network to get galaxy type, Here only two types. 

In [4]:
#standard libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.cm as cm
import sys
import os
#wcs is incompabible with newest numpy thus below not used 
#from astropy import wcs
#to access astronomical images in fits format
from astropy.io import fits
#torch functions
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
#sklearn helper functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score,f1_score, log_loss
#xgboost for comparison
from xgboost import XGBClassifier
#logistic regression for comparison 
from sklearn.linear_model import LogisticRegression

Getting the data. It are currently four fields. 

In [5]:
cutouts1=np.load("stripe82_1_ell_spiral_im.npy")
cutouts2=np.load("stripe82_2_ell_spiral_im.npy")
cutouts3=np.load("stripe82_3_ell_spiral_im.npy")
cutouts4=np.load("stripe82_4_ell_spiral_im.npy")
print(cutouts1.shape)
df1=pd.read_csv("stripe82_1_ell_spiral_table.csv")
df2=pd.read_csv("stripe82_2_ell_spiral_table.csv")
df3=pd.read_csv("stripe82_3_ell_spiral_table.csv")
df4=pd.read_csv("stripe82_4_ell_spiral_table.csv")
print(df1.columns)

(43, 43, 1, 168)
Index(['Unnamed: 0', 'index', 'objid', 'ra', 'dec', 'psfMag_u', 'psfMag_g',
       'psfMag_r', 'psfMag_i', 'psfMag_z', 'probPSF_u', 'probPSF_g',
       'probPSF_r', 'probPSF_i', 'probPSF_z', 'modelMag_u', 'modelMag_g',
       'modelMag_r', 'modelMag_i', 'modelMag_z', 'petroRad_g', 'petroRad_r',
       'petroRad_i', 'run', 'rerun', 'camcol', 'field', 'type', 'specobjid',
       'class', 'subclass', 'redshift', 'plate', 'mjd', 'fiberid', 'nvote',
       'p_el', 'p_cw', 'p_acw', 'p_edge', 'p_dk', 'p_mg', 'p_el_debiased',
       'p_cs_debiased', 'spiral', 'elliptical', 'uncertain', 'image',
       'pixel_x', 'pixel_y', 'off_image'],
      dtype='object')


Now I built the function to combine the four numpy arrays.

In [21]:
#combines numpy arrays of 4d shape, same shape first 3, last variable
def comb_nump_4d(input_list):
    l=0
    for i in range(len(input_list)):
        l+=input_list[i].shape[3]
    combined=np.zeros((input_list[0].shape[0],input_list[0].shape[1],input_list[0].shape[2],l))
    l=0
    for i in range(len(input_list)):
        combined[:,:,:,l:l+input_list[i].shape[3]]=input_list[i]
        l+=input_list[i].shape[3]  
    return combined

Combining the images. 

In [34]:
cutout_lists=[cutouts1,cutouts2,cutouts3,cutouts4]
cutouts=comb_nump_4d(cutout_lists)

Now combining the data frames with the classfications and more meta data. 

In [28]:
df=pd.concat([df1,df2,df3,df4],ignore_index=True)
print(df1.head(),df.head(),df4.tail(),df.tail())

   Unnamed: 0  index                objid         ra       dec  psfMag_u  \
0           0      1  1237663237128388701  50.130513 -1.228488  22.01211   
1           1    123  1237666299481817102  50.160628 -1.035026  19.01124   
2           2    140  1237663237128388949  50.167304 -1.241885  22.36878   
3           3    458  1237666300018557091  50.006004 -0.495751  21.98376   
4           4    110  1237663238739067202  50.387910  0.198944  21.86834   

   psfMag_g  psfMag_r  psfMag_i  psfMag_z  ...   p_mg  p_el_debiased  \
0  20.06700  18.86854  18.35853  17.93478  ...  0.032          0.935   
1  17.46116  16.85288  16.54355  16.19304  ...  0.000          0.971   
2  20.17246  18.94735  18.44240  17.99196  ...  0.000          0.964   
3  19.93291  18.73658  18.31895  17.89719  ...  0.000          0.893   
4  19.78908  18.63620  18.16101  17.63892  ...  0.018          0.755   

   p_cs_debiased  spiral  elliptical  uncertain  image      pixel_x  \
0          0.000       0           1   

Has worked, now looking on classes. 

In [30]:
print(df.spiral.value_counts())

1    504
0    206
Name: spiral, dtype: int64


Somewhat inbalanced, clearly more spirals than ellipctical galaxies. 

In [31]:
#adding cpu
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")



Using cpu device


In [36]:
target_train, target_test,image_train,image_test,df_train,df_test= train_test_split(df.loc[:,"spiral"],cutouts.T,df,train_size=0.60, shuffle=True, random_state=1)
print("shape of image train data")
print(image_train.shape)
print(image_test.shape)

shape of image train data
(426, 1, 43, 43)
(284, 1, 43, 43)


In [37]:
def pred_torch(model,data):
    y_pred_list_c = []
    with torch.no_grad():
        model.eval()
        for X_batch, _ in data:
            X_batch = X_batch.to(device)
            y_test_pred = model(X_batch)
            y_pred_list_c.append(y_test_pred.cpu().numpy())
    y_pred_list_c = [a.squeeze().tolist() for a in y_pred_list_c]
    return y_pred_list_c  

In [38]:
class ClassificationDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [39]:
target_train, target_test = np.array(target_train), np.array(target_test)

In [40]:
BATCH_SIZE=32

In [41]:
train_im_dataset = ClassificationDataset(torch.from_numpy(image_train).float(), torch.from_numpy(target_train).float())
test_im_dataset = ClassificationDataset(torch.from_numpy(image_test).float(), torch.from_numpy(target_test).float())


In [42]:
train_im_loader = DataLoader(dataset=train_im_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_im_loader = DataLoader(dataset=test_im_dataset, batch_size=1)
train_im_loader_pred = DataLoader(dataset=train_im_dataset, batch_size=1)

Test input outputs relations for network of 3 layers, it is still 3 *3 convolutional and 2 *2 maximuma.

In [43]:
input0 = torch.randn(1, 1, 43, 43)
b=torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0)
output0=b(input0)
print(f"first conv layer input: {input0.shape} output: {output0.shape}")

m = nn.MaxPool2d((2, 2), stride=(2, 2))
#standard drops but can be changed, can also use pooling and co get better number 
output1 = m(output0)
print(f"max pool input:{output0.shape} output:{output1.shape}")
#input format (Batch, Number Channels, height, width)
b2=torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0)

output2=b2(output1)
print(f"second conv layer input: {output1.shape} output: {output2.shape}")
output3 = m(output2)
print(f"second max pool layer input: {output2.shape} output: {output3.shape}")

b3=torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)

output4=b3(output3)
print(f"third conv layer input: {output3.shape} output: {output4.shape}")
output5 = m(output4)
print(f"third max pool layer input: {output4.shape} output: {output5.shape}")

first conv layer input: torch.Size([1, 1, 43, 43]) output: torch.Size([1, 16, 41, 41])
max pool input:torch.Size([1, 16, 41, 41]) output:torch.Size([1, 16, 20, 20])
second conv layer input: torch.Size([1, 16, 20, 20]) output: torch.Size([1, 32, 18, 18])
second max pool layer input: torch.Size([1, 32, 18, 18]) output: torch.Size([1, 32, 9, 9])
third conv layer input: torch.Size([1, 32, 9, 9]) output: torch.Size([1, 64, 7, 7])
third max pool layer input: torch.Size([1, 64, 7, 7]) output: torch.Size([1, 64, 3, 3])


In [44]:
class CNNBinary4(torch.nn.Module):
    #no padding because image does not really end when the data ends. 
    def __init__(self):
        super(CNNBinary4, self).__init__()
        # L1 ImgIn shape=(?, 43, 43, 1)
        # Conv -> (?, 41, 41, 16)
        # Pool -> (?, 20, 20, 16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=1 - keep_prob))
        # L2 ImgIn shape=(?, 20, 20, 16)
        # Conv      ->(?, 18, 18, 32)
        # Pool      ->(?, 9, 9, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=1 - keep_prob))
        # L3 ImgIn shape=(?, 9, 9, 32)
        # Conv      ->(?, 7, 7, 64)
        # Pool      ->(?, 3, 3, 64)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=1 - keep_prob))        
        # L3 FC 3x3x64 inputs -> 128 outputs
        self.fc1 = torch.nn.Linear(3 * 3 * 64, 128, bias=True)
        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - keep_prob))
        # L4 Final FC 128 inputs -> 1 output
        self.fc2 = torch.nn.Linear(128, 1, bias=True) #
        torch.nn.init.xavier_uniform_(self.fc2.weight) # initialize parameters

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out) #dont forget to add/omit layer here
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc1(out)
        out = torch.sigmoid(self.fc2(out))
        return out

In [49]:
keep_prob=1
model1 =CNNBinary4()
model1.to(device)
print(model1)

CNNBinary4(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (fc1): Linear(in_features=576, out_features=128, bias=True)
  (layer4): Sequential(
    (0): Linear(in_features=576, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0, inplace=False)
  )
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


/tmp/ipykernel_19938/994019738.py:31: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc1.weight)


In [50]:
#define the function to fit it
#parameters: model used, train_data, test_data, epchs, batch_size, learning_rate, file to collect sats, 
#optional regularization 
def torch_fit(model,train_loader,test_loader,epochs,batch_size,learning_rate,loss_stats,l2reg=0):
    learning_rate = learning_rate
    criterion = torch.nn.BCELoss()    # Softmax is internally computed.
    #if no regularization
    if l2reg==0:
        optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    #l2 regularization is added in optimizer as weight_decay=1e-5 or nsimilar 
    else:
        optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate,weight_decay=l2reg)        
    print("Begin training.")
    for e in tqdm(range(1, epochs+1)):
    
        # TRAINING
        train_epoch_loss = 0
        model.train()
        for X_train_batch, y_train_batch in train_loader:
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            optimizer.zero_grad()
        
            y_train_pred = model(X_train_batch)
        
            train_loss = criterion(y_train_pred, y_train_batch.unsqueeze(1))
        
            train_loss.backward()
            optimizer.step()
        
            train_epoch_loss += train_loss.item()
        
        
        # VALIDATION    
        with torch.no_grad():
        
            test_epoch_loss = 0
        
            model.eval()
            for X_test_batch, y_test_batch in test_loader:
                X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
            
                y_test_pred = model(X_test_batch)
                        
                test_loss = criterion(y_test_pred, y_test_batch.unsqueeze(1))
            
                test_epoch_loss += test_loss.item()
        loss_stats['train'].append(train_epoch_loss/len(train_loader))
        loss_stats['test'].append(test_epoch_loss/len(test_loader))                              
    
        print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Test Loss: {test_epoch_loss/len(test_loader):.5f}')    

In [ ]:
#somehow nothing improves like sometimes, unclear what is reason mistake in setup or real chance ? 
loss_stats_test = {
    'train': [], 'test': []
}
torch_fit(model1,train_im_loader,test_im_loader,200,32,0.01,loss_stats_test)

Begin training.


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 28.15375 | Test Loss: 27.46479
Epoch 002: | Train Loss: 29.06250 | Test Loss: 27.46479
Epoch 003: | Train Loss: 30.04464 | Test Loss: 27.46479
Epoch 004: | Train Loss: 30.53571 | Test Loss: 27.46479
Epoch 005: | Train Loss: 30.04464 | Test Loss: 27.46479
Epoch 006: | Train Loss: 29.06250 | Test Loss: 27.46479
Epoch 007: | Train Loss: 31.02679 | Test Loss: 27.46479
Epoch 008: | Train Loss: 30.04464 | Test Loss: 27.46479
Epoch 009: | Train Loss: 30.04464 | Test Loss: 27.46479
Epoch 010: | Train Loss: 29.06250 | Test Loss: 27.46479
Epoch 011: | Train Loss: 30.53571 | Test Loss: 27.46479
Epoch 012: | Train Loss: 29.55357 | Test Loss: 27.46479
Epoch 013: | Train Loss: 30.04464 | Test Loss: 27.46479
Epoch 014: | Train Loss: 29.55357 | Test Loss: 27.46479
Epoch 015: | Train Loss: 31.02679 | Test Loss: 27.46479
Epoch 016: | Train Loss: 28.57143 | Test Loss: 27.46479
Epoch 017: | Train Loss: 29.55357 | Test Loss: 27.46479
Epoch 018: | Train Loss: 30.53571 | Test Loss: 2

In [48]:
print(target_test,target_train.shape)

[1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 0
 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0
 1 0 0 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0
 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0
 1 1 1 1 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1
 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 1] (426,)


In [111]:
c_test=pred_torch(model1,test_im_loader)
c_train=pred_torch(model1,train_im_loader_pred)

In [112]:
print(c_train)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
